In [2]:
import os
import sys
import json

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["HADOOP_INSTALL"] = "/home/hadoop/hadoop"
os.environ["HADOOP_HOME"] = os.environ["HADOOP_INSTALL"]
os.environ["HADOOP_MAPRED_HOME"] = os.environ["HADOOP_INSTALL"]
os.environ["HADOOP_COMMON_HOME"] = os.environ["HADOOP_INSTALL"]
os.environ["HADOOP_HDFS_HOME"] = os.environ["HADOOP_INSTALL"]
os.environ["HADOOP_YARN_HOME"] = os.environ["HADOOP_INSTALL"]
os.environ["HADOOP_CONF_DIR"] = os.path.join(os.environ["HADOOP_INSTALL"], "/etc/hadoop")
os.environ["SPARK_HOME"] = "/home/hadoop/spark"
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], "python"))
sys.path.append(os.path.join(os.environ["SPARK_HOME"], "python/lib/py4j-0.10.9.2-src.zip"))

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("HelloLines") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "1024M") \
    .getOrCreate()
sc = spark.sparkContext
print(sc.uiWebUrl)

2023-10-07 20:20:32,614 WARN util.Utils: Your hostname, cloud resolves to a loopback address: 127.0.2.1; using 192.168.121.62 instead (on interface eth0)
2023-10-07 20:20:32,616 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-07 20:20:33,652 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-10-07 20:20:35,021 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2023-10-07 20:20:35,021 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2023-10-07 20:20:35,022 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


http://192.168.121.62:4043


In [5]:
rdd = sc.textFile("hdfs:/datasets/spotify/tracks.json")


In [6]:
rdd2 = rdd.map(lambda x: json.loads(x))
dfp = rdd2.toDF()

In [15]:
if ('q1.csv' not in os.listdir('hdfs')):
    min = dfp.agg({ 'duration_ms' : 'min' })
    avg = dfp.agg({ 'duration_ms' : 'avg' })
    max = dfp.agg({ 'duration_ms' : 'max' })
    aggregates = min.join(avg)
    aggregates = aggregates.join(max)
    aggregates.write.csv('q1.csv')
else:
    aggregates = spark.read.csv('q1.csv')
aggregates

In [12]:
display(aggregates)

DataFrame[min(duration_ms): bigint, avg(duration_ms): double, max(duration_ms): bigint]